In [1]:
import numpy as np
import pandas as pd

In [2]:
#텍스트 파일이 규칙적인 경우에는 table로 불러오면 됨
train_df = pd.read_table("./data/ratings_train.txt")
test_df = pd.read_table("./data/ratings_test.txt")

In [3]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
#null 및 숫자를 공백 문자로 변경
#대부분의 문자 변경은 가능하면 re 사용
#가능하면 축약 리스트 사용
#가능하면 for를 쓰지 않는 방향으로 진행

import re 
train_df = train_df.fillna(" ") #공값을 빈 문자열로 바꿔주지 않으면 람다식에서 오류가 남
train_df['document'].apply(lambda x: re.sub(r"\d+", " ", x)) #apply는 전체 행에 실행함, for 루프보다 빠름

test_df = test_df.fillna(" ")
test_df['document'].apply(lambda x: re.sub(r"\d+", " ", x))

0                                                      굳 ㅋ
1                                     GDNTOPCLASSINTHECLUB
2                    뭐야 이 평점들은.... 나쁘진 않지만  점 짜리는 더더욱 아니잖아
3                         지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4         D만 아니었어도 별 다섯 개 줬을텐데.. 왜  D로 나와서 제 심기를 불편하게 하죠??
                               ...                        
49995            오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996         의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997                   그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998       절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네
49999                                           마무리는 또 왜이래
Name: document, Length: 50000, dtype: object

In [5]:
#konlp는 자바와 연동할 수 있도록 해줌
#konlpy는 자바와 연동은 안 되지만 python에서 간편하게 작동함
from konlpy.tag import Okt
okt = Okt() #자바 기반이기 때문에 객체를 생성해줘야 함

In [6]:
def tw_tokenizer(text):
    tokenizer_ko = okt.morphs(text)
    return tokenizer_ko

In [7]:
tw_tokenizer("가난한 내가 나타샤를 사랑해서")

['가난한', '내', '가', '나타', '샤를', '사랑', '해서']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tw_tokenizer,
                        ngram_range=(1,2),
                        min_df=3,
                        max_df=0.9) #가다, 갔는데, 가다가 등을 어느 정도로 분리해서 볼 것인지(1은 전부 다 다르게 본다는 의미)

In [9]:
tfidf.fit(train_df['document'])

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<function tw_tokenizer at 0x000001E245C83D30>)

In [10]:
tfidf_matrix_train = tfidf.transform(train_df['document'])
tfidf_matrix_train

<150000x130333 sparse matrix of type '<class 'numpy.float64'>'
	with 3096525 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.linear_model import LogisticRegression
lg_clf = LogisticRegression(C=3.5, random_state=42) #민감도 3.5
lg_clf.fit(tfidf_matrix_train, train_df["label"])

LogisticRegression(C=3.5, random_state=42)

In [13]:
import joblib

In [14]:
joblib.dump(lg_clf, "movie_lr.pkl")

['movie_lr.pkl']

In [17]:
joblib.dump(tfidf, "movie_lr_tfidf.pkl")

['movie_lr_tfidf.pkl']

## 역직렬화

In [19]:
movie_lr = joblib.load("movie_lr.pkl")
movie_lr_tfidf = joblib.load("movie_lr_tfidf.pkl")

In [40]:
q = "좋아"
q = re.sub(r"\d+", " ", q)
q_matrix = movie_lr_tfidf.transform([q])

In [41]:
movie_lr.predict(q_matrix)

array([1], dtype=int64)